# coverage data download grid

We fit the client's requested spatial extent to a fixed grid, in order to potentially allow for caching

Below we define a bunch of bboxes, simulating input from the client. For each bbox, we try to fit it to our coverage preparation grid. This means enlarging the original bbox so that it aligns with the grid

If a bbox falls completely outside of the grid we raise an error. However, if the bbox partially intersects with the grid we clip it to the grid

In [1]:
import httpx
import ipyleaflet
import numpy as np
import shapely
import sidecar
import sqlmodel

from arpav_ppcv import (
    config,
    database,
    datadownloads,
    exceptions,
    operations,
)
from arpav_ppcv.thredds import (
    crawler,
    ncss,
)

settings = config.get_settings()
session = sqlmodel.Session(database.get_engine(settings))
http_client = httpx.AsyncClient()

cov = database.get_coverage(
    session, 
    (
        "tas_30yr_anomaly_seasonal_agree_model_ensemble-"
        "30yr-forecast-model_ensemble-tas-anomaly-rcp85-tw1-winter"
    )
)

arpav_grid = datadownloads.CoverageDownloadGrid.from_config(settings.coverage_download_settings.spatial_grid)

bboxes = {
    "inside": shapely.from_wkt("POLYGON ((13.3 45.2, 13.3 46.8, 12.1 46.8, 12.1 45.2, 13.3 45.2))"),
    "water": shapely.from_wkt("Polygon ((12.99700423944866223 45.29195993946695609, 13.35000423944866199 45.29195993946695609, 13.35000423944866199 45.43295993946695432, 12.99700423944866223 45.43295993946695432, 12.99700423944866223 45.29195993946695609))"),
    "tiny": shapely.from_wkt("Polygon ((12.815779996210372 46.20851850767168401, 12.83977999621037114 46.20851850767168401, 12.83977999621037114 46.2225185076716869, 12.815779996210372 46.2225185076716869, 12.815779996210372 46.20851850767168401))"),
    "outside_left": shapely.from_wkt("Polygon ((8.512064207837712 45.71491203133463443, 9.36806420783771188 45.71491203133463443, 9.36806420783771188 46.35891203133463279, 8.512064207837712 46.35891203133463279, 8.512064207837712 45.71491203133463443))"),
    "outside_right": shapely.from_wkt("Polygon ((14.22092928274156876 45.26076856403433624, 15.15092928274156847 45.26076856403433624, 15.15092928274156847 45.91976856403433516, 14.22092928274156876 45.91976856403433516, 14.22092928274156876 45.26076856403433624))"),
    "outside_top": shapely.from_wkt("Polygon ((11.3738158030779779 47.49323322467401454, 12.00281580307797746 47.49323322467401454, 12.00281580307797746 47.90323322467401113, 11.3738158030779779 47.90323322467401113, 11.3738158030779779 47.49323322467401454))"),
    "outside_bottom": shapely.from_wkt("Polygon ((10.81756741116426923 43.66521396617659434, 11.68156741116427 43.66521396617659434, 11.68156741116427 44.32421396617659326, 10.81756741116426923 44.32421396617659326, 10.81756741116426923 43.66521396617659434))"),
    "intersects_left": shapely.from_wkt("Polygon ((9.92464236098699892 45.29755871480751495, 10.55364236098699848 45.29755871480751495, 10.55364236098699848 45.68555871480751307, 9.92464236098699892 45.68555871480751307, 9.92464236098699892 45.29755871480751495))"),
    "intersects_right": shapely.from_wkt("Polygon ((13.74519052649958084 46.49788419209499324, 14.4991905264995804 46.49788419209499324, 14.4991905264995804 46.88588419209499136, 13.74519052649958084 46.88588419209499136, 13.74519052649958084 46.49788419209499324))"),
    "intersects_top": shapely.from_wkt("Polygon ((12.65465091603717696 46.82026106391365516, 13.31365091603717765 46.82026106391365516, 13.31365091603717765 47.37626106391365255, 12.65465091603717696 47.37626106391365255, 12.65465091603717696 46.82026106391365516))"),
    "intersects_bottom": shapely.from_wkt("Polygon ((12.8303083029572953 44.43399983300166411, 13.44530830295729551 44.43399983300166411, 13.44530830295729551 45.00499983300166207, 12.8303083029572953 45.00499983300166207, 12.8303083029572953 44.43399983300166411))"),   
}


In [2]:
fitted_bboxes = {}
for name, geom in bboxes.items():
    try:
        fitted = arpav_grid.fit_bbox(geom)
    except exceptions.CoverageDataRetrievalError:
        pass
    else:
        fitted_bboxes[f"fitted_{name}"] = fitted


In [3]:
map_center = arpav_grid.shapely_box.centroid
m = ipyleaflet.Map(center=(map_center.y, map_center.x), zoom=7)
m.add_control(ipyleaflet.LayersControl(position="topright"))

grid_layer = ipyleaflet.WKTLayer(
    name="grid", 
    wkt_string=arpav_grid.shapely_multipolygon.wkt, 
    style={
        "color": "blue",
        "fill": False,
        "weight": 1,
    }
)
m.add_layer(grid_layer)

wms_layer = ipyleaflet.WMSLayer(
    name=cov.identifier.partition("-")[0],
    url=f"https://arpav.geobeyond.dev/api/v2/coverages/wms/{cov.identifier}",
    layers="tas-uncertainty_group",
    format="image/png",
    transparent=True,
)
m.add_layer(wms_layer)

In [4]:
bbox_layers = [
    ipyleaflet.WKTLayer(
        name=name, 
        wkt_string=geom.wkt, 
        style={
            "color": "green",
            "dashArray": 8,
        }
    ) for geom in bboxes.values()
]
bbox_layers_group = ipyleaflet.LayerGroup(
    name="bboxes",
    layers=bbox_layers
)
fitted_bbox_layers = [
    ipyleaflet.WKTLayer(
        name=name, 
        wkt_string=geom.wkt, 
        style={
            "color": "red",
            "dashArray": 8,
        }
    ) for geom in fitted_bboxes.values()
]
fitted_bbox_layers_group = ipyleaflet.LayerGroup(
    name="fitted bboxes",
    layers=fitted_bbox_layers
)

m.add_layer(bbox_layers_group)
m.add_layer(fitted_bbox_layers_group)

In [5]:
sc = sidecar.Sidecar(title="map")

with sc:
    display(m)

In [6]:
crawler.get_thredds_url_fragment(cov, settings.thredds_server.base_url)

'ensembletwbc/std/clipped/tas_avgagree_anom_tw1_rcp85_DJF_VFVGTAA.nc'

In [7]:
async for chunk in ncss.async_query_dataset_area():

SyntaxError: incomplete input (3925259188.py, line 1)